In [1]:
import h5py
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
import scipy.stats as ss

from uoineuro.utils import calculate_selection_ratio

%matplotlib inline

In [2]:
base_path = os.path.join(os.environ['HOME'],
                         'fits/uoineuro/coupling')

In [3]:
# consolidate A1 results
a1_results_path = os.path.join(base_path, 'ecog/ecog_coupling.h5')
a1_results = h5py.File(a1_results_path, 'r')
# extract coupling coefficients
a1_lasso_ccs = np.median(a1_results['lasso/coupling_coefs'], axis=0)
a1_uoi_ccs = np.median(a1_results['uoi_lasso_bic/coupling_coefs'], axis=0)

In [32]:
# consolidate V1 results
v1_results_paths = [os.path.join(base_path, 'pvc11/pvc11_monkey1_coupling.h5'),
                    os.path.join(base_path, 'pvc11/pvc11_monkey2_coupling.h5'),
                    os.path.join(base_path, 'pvc11/pvc11_monkey3_coupling.h5')]
n_v1_fits = len(v1_results_paths)
v1_results = [h5py.File(v1_results_path, 'r')
              for v1_results_path in v1_results_paths]
# extract coupling coefficients
v1_lasso_ccs = [np.median(v1_result['glmnet_poisson/coupling_coefs'], axis=0)
                for v1_result in v1_results]
v1_uoi_ccs = [np.median(v1_result['uoi_poisson_bic/coupling_coefs'], axis=0)
              for v1_result in v1_results]

In [5]:
# consolidate M1 results
m1_results_paths = [os.path.join(base_path, 'nhp/nhp_indy_20160407_02_coupling.h5'),
                    os.path.join(base_path, 'nhp/nhp_indy_20160411_01_coupling.h5'),
                    os.path.join(base_path, 'nhp/nhp_indy_20160411_02_coupling.h5')]
n_m1_fits = len(m1_results_paths)
m1_results = [h5py.File(m1_results_path, 'r')
              for m1_results_path in m1_results_paths]
# extract coupling coefficients
m1_lasso_ccs = [np.median(m1_result['glmnet_poisson/coupling_coefs'], axis=0)
                for m1_result in m1_results]
m1_uoi_ccs = [np.median(m1_result['uoi_poisson_bic/coupling_coefs'], axis=0)
              for m1_result in m1_results]

## Selection Ratios

In [92]:
print('Auditory Cortex')
print('---------------')

ac_baseline_srs = calculate_selection_ratio(a1_lasso_ccs)
ac_uoi_srs = calculate_selection_ratio(a1_uoi_ccs)

# calculate medians and IQRs
ac_median_baseline_sr = np.median(ac_baseline_srs)
ac_iqr_baseline_sr = ss.iqr(ac_baseline_srs)
ac_median_uoi_sr = np.median(ac_uoi_srs)
ac_iqr_uoi_sr = ss.iqr(ac_uoi_srs)

# print out summary stats
print('Baseline: ', format(ac_median_baseline_sr, '0.4f'), '+/-', format(ac_iqr_baseline_sr / 2, '0.4f'))
print('UoI: ', format(ac_median_uoi_sr, '0.4f'), '+/-', format(ac_iqr_uoi_sr / 2, '0.4f'))
print('Fraction of Reduction:', format(ac_median_baseline_sr / ac_median_uoi_sr, '0.4f'))
print('WSRT: ', ss.wilcoxon(ac_baseline_srs, ac_uoi_srs))
print('Cohens d: ', np.mean(ac_baseline_srs - ac_uoi_srs) / np.std(ac_baseline_srs))

Auditory Cortex
---------------
Baseline:  0.2992 +/- 0.0679
UoI:  0.1339 +/- 0.0285
Fraction of Reduction: 2.2353
WSRT:  WilcoxonResult(statistic=6.0, pvalue=1.0880339977134602e-22)
Cohens d:  1.354441416455575


In [96]:
vc_baseline_srs = [calculate_selection_ratio(v1_lasso_cc)
                   for v1_lasso_cc in v1_lasso_ccs]
vc_uoi_srs = [calculate_selection_ratio(v1_uoi_cc)
              for v1_uoi_cc in v1_uoi_ccs]

for idx in range(n_v1_fits):
    baseline = vc_baseline_srs[idx]
    uoi = vc_uoi_srs[idx]
    print('Dataset %s:' %idx)
    print('-----------')
    print('Baseline: ', format(np.median(baseline), '0.4f'), '+/-', format(ss.iqr(baseline) / 2, '0.4f'))
    print('UoI: ', format(np.median(uoi), '0.4f'), '+/-', format(ss.iqr(uoi) / 2, '0.4f'))
    print('Reduction in features: ', np.median(baseline) / np.median(uoi))
    print('WSRT: ', ss.wilcoxon(baseline, uoi))
    print('Cohens d: ', np.mean(baseline - uoi) / np.std(baseline))
    print('\n')

Dataset 0:
-----------
Baseline:  0.5905 +/- 0.1071
UoI:  0.2667 +/- 0.0667
Reduction in features:  2.2142857142857144
WSRT:  WilcoxonResult(statistic=1.0, pvalue=4.078866893907982e-19)
Cohens d:  1.9346601684828189


Dataset 1:
-----------
Baseline:  0.6609 +/- 0.1710
UoI:  0.2586 +/- 0.1149
Reduction in features:  2.5555555555555554
WSRT:  WilcoxonResult(statistic=1.0, pvalue=3.845459358460536e-16)
Cohens d:  1.5752158138870465


Dataset 2:
-----------
Baseline:  0.5946 +/- 0.1002
UoI:  0.2793 +/- 0.0732
Reduction in features:  2.1290322580645165
WSRT:  WilcoxonResult(statistic=0.0, pvalue=4.0686381320283395e-20)
Cohens d:  2.408849343951098




In [97]:
mc_baseline_srs = [calculate_selection_ratio(m1_lasso_cc)
                   for m1_lasso_cc in m1_lasso_ccs]
mc_uoi_srs = [calculate_selection_ratio(m1_uoi_cc)
              for m1_uoi_cc in m1_uoi_ccs]

for idx in range(n_m1_fits):
    baseline = mc_baseline_srs[idx]
    uoi = mc_uoi_srs[idx]
    print('Dataset %s:' %idx)
    print('-----------')
    print('Baseline: ', format(np.median(baseline), '0.4f'), '+/-', format(ss.iqr(baseline) / 2, '0.4f'))
    print('UoI: ', format(np.median(uoi), '0.4f'), '+/-', format(ss.iqr(uoi) / 2, '0.4f'))
    print('Reduction in features: ', np.median(baseline) / np.median(uoi))
    print('WSRT: ', ss.wilcoxon(baseline, uoi))
    print('Cohens d: ', np.mean(baseline - uoi) / np.std(baseline))
    print('\n')

Dataset 0:
-----------
Baseline:  0.4333 +/- 0.1222
UoI:  0.0741 +/- 0.0380
Reduction in features:  5.8500000000000005
WSRT:  WilcoxonResult(statistic=0.0, pvalue=6.669083051880734e-24)
Cohens d:  1.8698271148250885


Dataset 1:
-----------
Baseline:  0.4552 +/- 0.1198
UoI:  0.0828 +/- 0.0414
Reduction in features:  5.5
WSRT:  WilcoxonResult(statistic=0.0, pvalue=6.878272638964072e-25)
Cohens d:  1.764650488003609


Dataset 2:
-----------
Baseline:  0.4589 +/- 0.1062
UoI:  0.0890 +/- 0.0411
Reduction in features:  5.153846153846154
WSRT:  WilcoxonResult(statistic=0.0, pvalue=2.2069942157533432e-25)
Cohens d:  1.9309963615229062




## Predictive Performance

In [98]:
ac_baseline_pp = np.median(a1_results['lasso/r2s_test'][:], axis=0)
ac_uoi_pp = np.median(a1_results['uoi_lasso_bic/r2s_test'][:], axis=0)

print('Baseline PP: ', np.median(ac_baseline_pp), '+/-', ss.iqr(ac_baseline_pp))
print('UoI PP: ', np.median(ac_uoi_pp), '+/-', ss.iqr(ac_uoi_pp))
print('Median del PP: ', np.median(ac_baseline_pp - ac_uoi_pp))
print('Wilcoxon Sign Rank Test: ', ss.wilcoxon(x=ac_baseline_pp,
                                               y=ac_uoi_pp))
print('Cohens D: ', np.mean(ac_baseline_pp - ac_uoi_pp) / np.std(ac_baseline_pp))

Baseline PP:  0.9769597427791629 +/- 0.04153534270144987
UoI PP:  0.9768421413247849 +/- 0.04264205212336036
Median del PP:  4.630757180323908e-05
Wilcoxon Sign Rank Test:  WilcoxonResult(statistic=2746.0, pvalue=0.0010140364137717843)
Cohens D:  0.004911669739322192


In [82]:
vc_baseline_pp = [np.median(v1_results[idx]['glmnet_poisson/deviances_test'][:], axis=0) for idx in range(n_v1_fits)]
vc_uoi_pp = [np.median(v1_results[idx]['uoi_poisson_log/deviances_test'][:], axis=0) for idx in range(n_v1_fits)]

for dataset in range(n_v1_fits):
    baseline = vc_baseline_pp[dataset]
    uoi = vc_uoi_pp[dataset]
    
    print('Dataset %s:' %dataset)
    print('-----------')
    print('Baseline BIC: ', format(np.median(baseline), '0.2f'), '+/-', format(ss.iqr(baseline), '0.2f'))
    print('UoI BIC: ', format(np.median(uoi), '0.2f'), '+/-', format(ss.iqr(uoi), '0.2f'))
    print('Median del BIC: ', format(np.median(baseline - uoi), '0.2f'))
    print('Wilcoxon Sign Rank Test: ',
          ss.wilcoxon(x=vc_baseline_bics[idx],
                      y=vc_uoi_bics[idx]))
    print('Cohens D: ', np.mean(baseline - uoi) / np.std(baseline))
    print('\n')

Dataset 0:
-----------
Baseline BIC:  202.89 +/- 69.39
UoI BIC:  205.06 +/- 71.04
Median del BIC:  -2.99
Wilcoxon Sign Rank Test:  WilcoxonResult(statistic=0.0, pvalue=4.096357955947718e-20)
Cohens D:  -0.04625702029560614


Dataset 1:
-----------
Baseline BIC:  165.69 +/- 87.04
UoI BIC:  167.87 +/- 86.81
Median del BIC:  -1.97
Wilcoxon Sign Rank Test:  WilcoxonResult(statistic=0.0, pvalue=4.096357955947718e-20)
Cohens D:  -0.03283287013248259


Dataset 2:
-----------
Baseline BIC:  212.86 +/- 82.95
UoI BIC:  217.40 +/- 82.40
Median del BIC:  -3.31
Wilcoxon Sign Rank Test:  WilcoxonResult(statistic=0.0, pvalue=4.096357955947718e-20)
Cohens D:  -0.0472620057877084




In [88]:
mc_baseline_pp = [np.median(m1_results[idx]['glmnet_poisson/deviances_test'][:], axis=0) for idx in range(n_m1_fits)]
mc_uoi_pp = [np.median(m1_results[idx]['uoi_poisson_log/deviances_test'][:], axis=0) for idx in range(n_m1_fits)]

for dataset in range(n_m1_fits):
    baseline = np.nan_to_num(mc_baseline_pp[dataset])
    uoi = np.nan_to_num(mc_uoi_pp[dataset])
    
    print('Dataset %s:' %dataset)
    print('-----------')
    print('Baseline BIC: ', format(np.median(baseline), '0.2f'), '+/-', format(ss.iqr(baseline), '0.2f'))
    print('UoI BIC: ', format(np.median(uoi), '0.2f'), '+/-', format(ss.iqr(uoi), '0.2f'))
    print('Median del BIC: ', format(np.median(baseline - uoi), '0.2f'))
    print('Wilcoxon Sign Rank Test: ',
          ss.wilcoxon(x=vc_baseline_bics[idx],
                      y=vc_uoi_bics[idx]))
    print('Cohens D: ', np.mean(baseline - uoi) / np.std(baseline))
    print('\n')

Dataset 0:
-----------
Baseline BIC:  231.66 +/- 129.12
UoI BIC:  233.28 +/- 129.16
Median del BIC:  -2.47
Wilcoxon Sign Rank Test:  WilcoxonResult(statistic=0.0, pvalue=4.096357955947718e-20)
Cohens D:  -0.03009294933633112


Dataset 1:
-----------
Baseline BIC:  263.01 +/- 120.06
UoI BIC:  266.26 +/- 117.27
Median del BIC:  -2.27
Wilcoxon Sign Rank Test:  WilcoxonResult(statistic=0.0, pvalue=4.096357955947718e-20)
Cohens D:  -0.025037853371152427


Dataset 2:
-----------
Baseline BIC:  248.21 +/- 113.79
UoI BIC:  249.80 +/- 115.33
Median del BIC:  -2.72
Wilcoxon Sign Rank Test:  WilcoxonResult(statistic=0.0, pvalue=4.096357955947718e-20)
Cohens D:  -0.0298387990130159




## Bayesian Information Criterion

In [71]:
ac_baseline_bics = np.median(a1_results['lasso/bics'][:], axis=0)
ac_uoi_bics = np.median(a1_results['uoi_lasso_bic/bics'][:], axis=0)

print('Baseline BIC: ', format(np.median(ac_baseline_bics), '0.2f'), '+/-', format(ss.iqr(ac_baseline_bics), '0.2f'))
print('UoI BIC: ', format(np.median(ac_uoi_bics), '0.2f'), '+/-', format(ss.iqr(ac_uoi_bics), '0.2f'))
print('Median del BIC: ', format(np.median(ac_baseline_bics - ac_uoi_bics), '0.2f'))
print('Wilcoxon Sign Rank Test: ',
      ss.wilcoxon(x=ac_baseline_bics,
                  y=ac_uoi_bics))

Baseline BIC:  -6239.96 +/- 3768.85
UoI BIC:  -6490.60 +/- 3718.17
Median del BIC:  170.01
Wilcoxon Sign Rank Test:  WilcoxonResult(statistic=0.0, pvalue=9.513002484100111e-23)


In [65]:
vc_baseline_bics = [np.median(v1_results[idx]['glmnet_poisson/bics'][:], axis=0) for idx in range(n_v1_fits)]
vc_uoi_bics = [np.median(v1_results[idx]['uoi_poisson_log/bics'][:], axis=0) for idx in range(n_v1_fits)]

for dataset in range(n_v1_fits):
    baseline = vc_baseline_bics[dataset]
    uoi = vc_uoi_bics[dataset]
    
    print('Dataset %s:' %dataset)
    print('-----------')
    print('Baseline BIC: ', format(np.median(baseline), '0.2f'), '+/-', format(ss.iqr(baseline), '0.2f'))
    print('UoI BIC: ', format(np.median(uoi), '0.2f'), '+/-', format(ss.iqr(uoi), '0.2f'))
    print('Median del BIC: ', format(np.median(baseline - uoi), '0.2f'))
    print('Wilcoxon Sign Rank Test: ',
          ss.wilcoxon(x=vc_baseline_bics[idx],
                      y=vc_uoi_bics[idx]))
    print('\n')

Dataset 0:
-----------
Baseline BIC:  -15349.34 +/- 41893.35
UoI BIC:  -15516.19 +/- 41941.48
Median del BIC:  149.14
Wilcoxon Sign Rank Test:  WilcoxonResult(statistic=0.0, pvalue=4.096357955947718e-20)


Dataset 1:
-----------
Baseline BIC:  -2864.02 +/- 22093.74
UoI BIC:  -3007.10 +/- 22174.05
Median del BIC:  131.25
Wilcoxon Sign Rank Test:  WilcoxonResult(statistic=0.0, pvalue=4.096357955947718e-20)


Dataset 2:
-----------
Baseline BIC:  -35164.55 +/- 68138.67
UoI BIC:  -35309.01 +/- 68132.88
Median del BIC:  161.33
Wilcoxon Sign Rank Test:  WilcoxonResult(statistic=0.0, pvalue=4.096357955947718e-20)




In [66]:
mc_baseline_bics = [np.median(m1_results[idx]['glmnet_poisson/bics'][:], axis=0) for idx in range(n_m1_fits)]
mc_uoi_bics = [np.median(m1_results[idx]['uoi_poisson_log/bics'][:], axis=0) for idx in range(n_m1_fits)]

for dataset in range(n_m1_fits):
    baseline = mc_baseline_bics[dataset]
    uoi = mc_uoi_bics[dataset]
    
    print('Dataset %s:' %dataset)
    print('-----------')
    print('Baseline BIC: ', format(np.median(baseline), '0.2f'), '+/-', format(ss.iqr(baseline), '0.2f'))
    print('UoI BIC: ', format(np.median(uoi), '0.2f'), '+/-', format(ss.iqr(uoi), '0.2f'))
    print('Median del BIC: ', format(np.median(baseline - uoi), '0.2f'))
    print('Wilcoxon Sign Rank Test: ',
          ss.wilcoxon(x=mc_baseline_bics[idx],
                      y=mc_uoi_bics[idx]))
    print('\n')

Dataset 0:
-----------
Baseline BIC:  480.98 +/- 248.10
UoI BIC:  321.75 +/- 118.98
Median del BIC:  162.17
Wilcoxon Sign Rank Test:  WilcoxonResult(statistic=180.0, pvalue=6.058087006705419e-24)


Dataset 1:
-----------
Baseline BIC:  542.79 +/- 300.33
UoI BIC:  354.80 +/- 150.70
Median del BIC:  186.10
Wilcoxon Sign Rank Test:  WilcoxonResult(statistic=180.0, pvalue=6.058087006705419e-24)


Dataset 2:
-----------
Baseline BIC:  564.84 +/- 240.77
UoI BIC:  362.11 +/- 134.84
Median del BIC:  190.53
Wilcoxon Sign Rank Test:  WilcoxonResult(statistic=180.0, pvalue=6.058087006705419e-24)




## Non-zero Parameters

In [16]:
print('Auditory Cortex')
print('---------------')

# unravel all coefficients
a1_baseline_ccs_all = a1_lasso_ccs.ravel()
a1_uoi_ccs_all = a1_uoi_ccs.ravel()

# calculate non-zero coefficients
a1_baseline_nz_ccs = a1_baseline_ccs_all[a1_baseline_ccs_all != 0]
a1_uoi_nz_ccs = a1_uoi_ccs_all[a1_uoi_ccs_all != 0]

# perform KS test
print('KS 2-sample test: ', ss.ks_2samp(a1_baseline_nz_ccs, a1_uoi_nz_ccs).pvalue)

Auditory Cortex
---------------
KS 2-sample test:  5.88596274373403e-69


In [42]:
print('Visual Cortex')
print('-------------')


for dataset in range(n_v1_fits):
    v1_baseline_ccs_all = v1_lasso_ccs[dataset].ravel()
    v1_uoi_ccs_all = v1_uoi_ccs[dataset].ravel()
    
    v1_baseline_nz_ccs = v1_baseline_ccs_all[v1_baseline_ccs_all != 0]
    v1_uoi_nz_ccs = v1_uoi_ccs_all[v1_uoi_ccs_all != 0]

    print('Dataset %s:' %dataset,
          ss.ks_2samp(v1_baseline_nz_ccs, v1_uoi_nz_ccs).pvalue)

Visual Cortex
-------------
Dataset 0: 1.0
Dataset 1: 7.052979493638599e-60
Dataset 2: 1.0


In [41]:
v1_baseline_ccs_all = np.concatenate(
    [v1_lasso_ccs[idx].ravel()
     for idx in range(n_v1_fits)])
v1_uoi_ccs_all = np.concatenate(
    [v1_uoi_ccs[idx].ravel()
     for idx in range(n_v1_fits)])

v1_baseline_nz_ccs = v1_baseline_ccs_all[v1_baseline_ccs_all != 0]
v1_uoi_nz_ccs = v1_uoi_ccs_all[v1_uoi_ccs_all != 0]

print(ss.ks_2samp(v1_baseline_nz_ccs, v1_uoi_nz_ccs))

Ks_2sampResult(statistic=0.23965492673534994, pvalue=6.013057238389484e-293)


In [40]:
print('Motor Cortex')
print('-------------')

for dataset in range(n_m1_fits):
    m1_baseline_ccs_all = m1_lasso_ccs[dataset].ravel()
    m1_uoi_ccs_all = m1_uoi_ccs[dataset].ravel()
    
    m1_baseline_nz_ccs = m1_baseline_ccs_all[m1_baseline_ccs_all != 0]
    m1_uoi_nz_ccs = m1_uoi_ccs_all[m1_uoi_ccs_all != 0]

    print('Dataset %s:' %dataset,
          ss.ks_2samp(m1_baseline_nz_ccs, m1_uoi_nz_ccs))

Motor Cortex
-------------
Dataset 0: Ks_2sampResult(statistic=0.42818217002104114, pvalue=1.9329147863565506e-204)
Dataset 1: Ks_2sampResult(statistic=0.39066074887674807, pvalue=2.570866913273217e-204)
Dataset 2: Ks_2sampResult(statistic=0.3827707226791267, pvalue=1.1356087074758565e-203)


In [39]:
m1_baseline_ccs_all = np.concatenate(
    [m1_lasso_ccs[idx].ravel()
     for idx in range(n_v1_fits)])
m1_uoi_ccs_all = np.concatenate(
    [m1_uoi_ccs[idx].ravel()
     for idx in range(n_v1_fits)])

m1_baseline_nz_ccs = m1_baseline_ccs_all[m1_baseline_ccs_all != 0]
m1_uoi_nz_ccs = m1_uoi_ccs_all[m1_uoi_ccs_all != 0]

print(ss.ks_2samp(m1_baseline_nz_ccs, m1_uoi_nz_ccs))

Ks_2sampResult(statistic=0.38891828386778193, pvalue=0.0)
